In [1]:
import pandas as pd
from difflib import get_close_matches as gcm

all_skills = 'skills/Skills.xlsx'
digital_skills = 'skills/Digital Skills.xlsx'
coursera_skills = 'coursera_skills/Coursera Skills To DB.xlsx'

In [2]:
def extract_skills(file_name):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    all_skills = []
    for key, df in dfs.items():
        all_skills.extend(df['Skill'].unique().tolist())
    df_skills = pd.DataFrame({'skill': all_skills})
    return df_skills.drop_duplicates()

def parent_check(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    for key, df in dfs.items():
        df_error = df.loc[~df['Parent'].isin(df_skills['skill'])].dropna()
        print(df_error['Parent'].unique().tolist())

def extract_relation(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    df_rel = pd.DataFrame(columns=['Skill', 'Parent'])
    skill_to_id_dict = df_skills.set_index('skill').to_dict()['skill_id']
    for key, df in dfs.items():
        df_rel = df_rel.append(df.replace(skill_to_id_dict))
    df_rel = df_rel.reset_index(drop=True)
    df_rel = df_rel.dropna()
    df_rel = df_rel.drop_duplicates()
    df_rel['Parent'] = df_rel['Parent'].astype(int)
    return df_rel

### Extract Skills

In [3]:
# All skills
df_s1 = extract_skills(all_skills)
df_s1.head()

,skill
0,Applied Science
1,Art
2,Business
3,Computer Science
4,Data Science


In [4]:
df_s1.describe(include='object')

,skill
count,2260
unique,2260
top,Feature Detection
freq,1


In [5]:
# Digital skills
df_s2 = extract_skills(digital_skills)
df_s2.head()

,skill
0,Algebra
1,Algorithm
2,Analytics
3,Artificial Intelligence (AI)
4,Big Data


In [6]:
df_s2.describe(include='object')

,skill
count,1009
unique,1009
top,Parsing
freq,1


In [7]:
# Combine
df_skills = df_s1.append(df_s2)
df_skills = df_skills.drop_duplicates()
df_skills = df_skills.dropna()
df_skills.describe()

,skill
count,3065
unique,3065
top,Feature Detection
freq,1


In [8]:
df_skills['skill_id'] = df_skills.index + 1
# df_skills['skill_id'] = df_skills.groupby('skill').grouper.group_info[0] + 1
# df_skills = df_skills.reset_index(drop=True)
df_skills = df_skills[['skill_id', 'skill']]
df_skills.head()

,skill_id,skill
0,1,Applied Science
1,2,Art
2,3,Business
3,4,Computer Science
4,5,Data Science


In [9]:
df_skills.describe(include='all')

,skill_id,skill
count,3065.000000,3065
unique,NaN,3065
top,NaN,Feature Detection
freq,NaN,1
mean,1069.966069,NaN
std,730.453695,NaN
min,1.000000,NaN
25%,459.000000,NaN
50%,911.000000,NaN
75%,1652.000000,NaN


### Coursera Skills

In [10]:
df_c = pd.read_excel(coursera_skills)
df_c.head()

,skill_id,c_skill,Skill,Parent
0,457,Low Poly,Low Poly,3D Computer Graphics
1,462,Polygon Mesh,Polygon Mesh,3D Computer Graphics
2,466,Uv Mapping,Uv Mapping,3D Computer Graphics
3,467,Framing,Framing,3D Computer Graphics
4,469,3D Graphics Software,3D Graphics Software,3D Computer Graphics


In [11]:
df_coursera = df_c[['skill_id', 'c_skill']]
df_coursera = df_coursera.rename(columns={'c_skill': 'skill'})
df_coursera.head()

,skill_id,skill
0,457,Low Poly
1,462,Polygon Mesh
2,466,Uv Mapping
3,467,Framing
4,469,3D Graphics Software


In [12]:
df_others = df_c[['Skill']].rename(columns={'Skill': 'skill'})
df_others = df_others.drop_duplicates()
df_others = df_others.dropna()
df_others['skill_id'] = df_others.index + df_skills['skill_id'].max()
df_others.head()

,skill,skill_id
0,Low Poly,2600
1,Polygon Mesh,2601
2,Uv Mapping,2602
3,Framing,2603
4,3D Graphics Software,2604


In [13]:
# Combine
df_all = df_skills.append(df_others)
df_all = df_all.drop_duplicates(subset=['skill'])
df_all = df_all.reset_index(drop=True)
df_all = df_all[['skill_id', 'skill']]
df_all.head()

,skill_id,skill
0,1,Applied Science
1,2,Art
2,3,Business
3,4,Computer Science
4,5,Data Science


In [14]:
df_all.describe(include='all')

,skill_id,skill
count,9470.000000,9470
unique,NaN,9470
top,NaN,Feature Detection
freq,NaN,1
mean,4692.079831,NaN
std,3062.240481,NaN
min,1.000000,NaN
25%,1736.250000,NaN
50%,4794.500000,NaN
75%,7416.500000,NaN


### Data (Parent) Checking/Cleaning

In [15]:
parent_check(all_skills, df_skills)

[]
[]
[]
[]
[]
[]
[]
[]
[]


In [16]:
parent_check(digital_skills, df_skills)

[]
[]
[]
[]
[]
[]


In [17]:
parent_check(coursera_skills, df_all)

[]


### Data (Duplicate) Checking/Cleaning

In [18]:
def skill_dup_check(reverse=False):
    skill_list = df_skills['skill'].tolist()
    pos_dups = {}
    for i in range(len(skill_list)):
        if reverse:
            skill = skill_list[-(i+1)]
            remaining = skill_list[:-(i+1)]
        else:
            skill = skill_list[i]
            remaining = skill_list[i+1:]
        pos_dup = gcm(skill, remaining, cutoff=0.9)
        if len(pos_dup) > 0:
            pos_dups[skill] = pos_dup
    return pos_dups

In [19]:
pos_dups = skill_dup_check()

for s in pos_dups:
    print('{}: {}'.format(s, pos_dups[s]))

Computer Graphics: ['3D Computer Graphics', '2D Computer Graphics']
Electrical Engineering: ['Electronic Engineering']
Aeronautical Engineering: ['Astronautical Engineering']
Macroeconomics: ['Microeconomics']
Market Research: ['Marketing Research']
Python: ['IPython']
Qualitative Marketing Research: ['Quantitative Marketing Research']
Sports Engineering: ['Support Engineering']
Social Geography: ['Soil Geography']
Statistical Analysis: ['SAS Statistical Analysis']
C Syntax: ['C# Syntax']
Supply Chain Risk Management: ['Supply Chain Management']
Web Application Security: ['Application Security', 'GIAC Web Application Security']
Column-oriented Database Management System: ['Document-oriented Database Management System']
Convolutional Neural Network (CNN): ['Convolutional Neural Network (ConvNet)']
General Linear Model: ['Generalized Linear Model']
Normal Distribution: ['Log-Normal Distribution']
Shift Invariant Artificial Neural Network (SIANN): ['Space Invariant Artificial Neural Netwo

In [20]:
pos_dups = skill_dup_check(reverse=True)

for s in pos_dups:
    print('{}: {}'.format(s, pos_dups[s]))

UC Browser: ['CM Browser']
Statistical Modeling: ['Statistical Modelling', 'Statistical Model']
Semi-supervised Machine Learning: ['Supervised Machine Learning']
Log-Normal Distribution: ['Normal Distribution']
CATS Applicant Tracking System: ['Applicant Tracking System']
IPython: ['Python']
GIAC Web Application Security: ['Web Application Security']
C# Syntax: ['C Syntax']
3D Computer Graphics: ['2D Computer Graphics', 'Computer Graphics']
2D Computer Graphics: ['Computer Graphics']
SAS Statistical Analysis: ['Statistical Analysis']
Quantitative Data Analysis: ['Qualitative Data Analysis']
Business Intelligence Development Studio: ['Business Intelligence Development']
Supply Chain Management: ['Supply Chain Risk Management']
International Trade Law: ['International Trade']
Last-in First-out (LIFO) Accounting: ['First-in First-out (FIFO) Accounting']
Unsupervised Machine Learning: ['Supervised Machine Learning']
Unsupervised Learning: ['Supervised Learning']
Space Invariant Artificial 

### Extract Skill Relation

In [21]:
# Skills
df_r1 = extract_relation(all_skills, df_skills)
df_r2 = extract_relation(digital_skills, df_skills)
df_rel = df_r1.append(df_r2)
df_rel = df_rel.drop_duplicates()
df_rel = df_rel.reset_index(drop=True)
df_rel = df_rel.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel.head()

,skill_1_id,skill_2_id
0,11,14
1,15,14
2,19,4
3,19,13
4,20,3


In [22]:
df_rel.describe(include='all')

,skill_1_id,skill_2_id
count,3766.0,3766.000000
unique,2322.0,NaN
top,131.0,NaN
freq,8.0,NaN
mean,NaN,608.857408
std,NaN,662.399057
min,NaN,1.000000
25%,NaN,103.000000
50%,NaN,328.000000
75%,NaN,954.250000


In [23]:
# Include coursera skills
df_r3 = extract_relation(coursera_skills, df_all)
# Combine
df_rel_all = df_rel.append(df_r3[['Skill', 'Parent']])
df_rel_all = df_rel_all.drop_duplicates()
df_rel_all = df_rel_all.reset_index(drop=True)
df_rel_all = df_rel_all.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel_all.head()

,skill_1_id,skill_2_id,skill_1_id,skill_2_id
0,11,14.0,NaN,NaN
1,15,14.0,NaN,NaN
2,19,4.0,NaN,NaN
3,19,13.0,NaN,NaN
4,20,3.0,NaN,NaN


In [24]:
df_rel_all.describe(include='all')

,skill_1_id,skill_2_id,skill_1_id,skill_2_id
count,3766.0,3766.000000,2348.0,2348.000000
unique,2322.0,NaN,2326.0,NaN
top,131.0,NaN,118.0,NaN
freq,8.0,NaN,2.0,NaN
mean,NaN,608.857408,NaN,835.054940
std,NaN,662.399057,NaN,1300.011553
min,NaN,1.000000,NaN,1.000000
25%,NaN,103.000000,NaN,90.000000
50%,NaN,328.000000,NaN,159.000000
75%,NaN,954.250000,NaN,1271.000000


### Extract Coursera Relation

In [25]:
df_crel = df_c[['skill_id', 'Skill']].rename(columns={'skill_id': 'coursera_skill_id', 'Skill': 'skill'})\
    .merge(df_all, how='left').drop(columns=['skill'])
df_crel.head()

,coursera_skill_id,skill_id
0,457,2600
1,462,2601
2,466,2602
3,467,2603
4,469,2604


In [26]:
df_crel.describe(include='all')

,coursera_skill_id,skill_id
count,7301.000000,7301.000000
mean,3650.941652,5769.890974
std,2107.856673,2636.810402
min,0.000000,2.000000
25%,1826.000000,3801.000000
50%,3651.000000,6009.000000
75%,5476.000000,7999.000000
max,7301.000000,9900.000000


### Export

In [27]:
df_skills.to_csv('skills_db/skill.csv', index=False)
df_rel.to_csv('skills_db/skill_tree.csv', index=False)
df_all.to_csv('skills_db/all_skill.csv', index=False)
df_rel_all.to_csv('skills_db/all_skill_tree.csv', index=False)
df_coursera.to_csv('skills_db/coursera_skill.csv', index=False)
df_crel.to_csv('skills_db/coursera_map.csv', index=False)